In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/09 16:13:58 WARN Utils: Your hostname, Macintosh.local resolves to a loopback address: 127.0.0.1; using 10.12.239.35 instead (on interface en0)
22/09/09 16:13:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/09 16:13:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Read the given datasets

In [4]:
ori_transaction1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot').sort('order_datetime')
ori_transaction2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot').sort('order_datetime')
 #read the parquet ori_transaction
ori_transaction = ori_transaction1.union(ori_transaction2)

In [3]:
tbl_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')

In [5]:
consumer_detail = spark.read.parquet('../data/tables/consumer_user_details.parquet')

In [6]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")

In [8]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")
tbl_consumer.count()

499999

# Merchant datasets

In [52]:
from pyspark.sql.functions import round
grouped_transaction = ori_transaction.groupBy('merchant_abn').agg(F.sum('dollar_value').alias('Amount'), F.count('dollar_value').alias('Count')).sort('merchant_abn')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction = grouped_transaction.withColumn('Monthly Amount', round(grouped_transaction['Amount']/12, 2))
#grouped_transaction = grouped_transaction.withColumn('Round Monthly Amount', round(grouped_transaction['Monthly Amount'], 2))
grouped_transaction = grouped_transaction.withColumn('Monthly Count', round(grouped_transaction['Count']/12, 2))

In [8]:
grouped_transaction_pd = grouped_transaction.toPandas()
tbl_merchants_pd = tbl_merchants.toPandas()
merchant = pd.merge(grouped_transaction_pd, tbl_merchants_pd)
merchant

22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:00:54 WARN RowBasedKeyValueBatch: Calling spill() on

,merchant_abn,avg,sum,count,name,tags
0,10023283211,215.909270,4.011594e+05,1858,Felis Limited,"((furniture, home furnishings and equipment sh..."
1,10142254217,38.781462,6.635508e+04,1711,Arcu Ac Orci Corporation,"([cable, satellite, and otHer pay television a..."
2,10165489824,15786.793362,3.157359e+04,2,Nunc Sed Company,"([jewelry, watch, clock, and silverware shops]..."
3,10187291046,114.591613,2.257455e+04,197,Ultricies Dignissim Lacus Foundation,"([wAtch, clock, and jewelry repair shops], [b]..."
4,10192359162,455.329192,9.926176e+04,218,Enim Condimentum PC,"([music shops - musical instruments, pianos, a..."
...,...,...,...,...,...,...
4014,99938978285,29.841844,3.015518e+05,10105,Elit Dictum Eu Ltd,"[(opticians, optical goods, and eyeglasses), (..."
4015,99974311662,276.702797,2.102941e+04,76,Mollis LLP,"((books, periodicals, and newspapers), (b), (t..."
4016,99976658299,149.873298,2.006354e+06,13387,Sociosqu Corp.,"((shoe shops), (a), (take rate: 6.57))"
4017,99987905597,344.174242,3.131986e+04,91,Commodo Hendrerit LLC,"[[motor vehicle Supplies and new parts], [a], ..."


In [9]:
for i in range(int(merchant['tags'].count())):
    merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


/var/folders/63/097df8mn6r1d0btnhwhjpkr40000gn/T/ipykernel_90056/3340515154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


In [10]:
#merchant['tags'].str.split(pat, n=-1, expand=False)
# to split into multiple columns by delimiter
merchant_tags = merchant['tags'].str.split(')', expand=True)

In [11]:
for row in range(int(len(merchant_tags))):
    for col in range(3):
        merchant_tags.iloc[row,col] = merchant_tags.iloc[row,col].replace(r'((', r'').replace(r', (', r'').replace(r'take rate:', r'')
merchant_tags.rename(columns = {0 : 'Store type', 1 : 'Revenue levels', 2 : 'Take rate'}, inplace = True)
merchant_tags = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]

In [12]:
merchant[['Store type', 'Revenue levels', 'Take rate']] = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]
merchant.drop(columns=['tags'])

,merchant_abn,avg,sum,count,name,Store type,Revenue levels,Take rate
0,10023283211,215.909270,4.011594e+05,1858,Felis Limited,"furniture, home furnishings and equipment shop...",e,0.18
1,10142254217,38.781462,6.635508e+04,1711,Arcu Ac Orci Corporation,"cable, satellite, and otHer pay television and...",b,4.22
2,10165489824,15786.793362,3.157359e+04,2,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",b,4.40
3,10187291046,114.591613,2.257455e+04,197,Ultricies Dignissim Lacus Foundation,"wAtch, clock, and jewelry repair shops",b,3.29
4,10192359162,455.329192,9.926176e+04,218,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",a,6.33
...,...,...,...,...,...,...,...,...
4014,99938978285,29.841844,3.015518e+05,10105,Elit Dictum Eu Ltd,"opticians, optical goods, and eyeglasses",b,4.50
4015,99974311662,276.702797,2.102941e+04,76,Mollis LLP,"books, periodicals, and newspapers",b,3.17
4016,99976658299,149.873298,2.006354e+06,13387,Sociosqu Corp.,shoe shops,a,6.57
4017,99987905597,344.174242,3.131986e+04,91,Commodo Hendrerit LLC,motor vehicle Supplies and new parts,a,6.82


# Consumer datasets

In [13]:
consumer_detail_pd = consumer_detail.toPandas()
tbl_consumer_pd = tbl_consumer.toPandas()
tbl_consumer_pd['consumer_id']=tbl_consumer_pd['consumer_id'].astype(int)
consumer = pd.merge(tbl_consumer_pd, consumer_detail_pd)
consumer

,name,address,state,postcode,gender,consumer_id,user_id
0,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1195503,1
1,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,2
2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,3
3,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,154128,4
4,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,712975,5
...,...,...,...,...,...,...,...
499994,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female,1385608,499995
499995,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed,1466964,499996
499996,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed,1253484,499997
499997,Donna Sutton,54140 Jacob Point,VIC,3989,Female,175005,499998


In [14]:
grouped_transaction = ori_transaction.groupBy('user_id').agg(F.avg('dollar_value').alias('avg'), F.sum('dollar_value').alias('sum'), F.count('dollar_value').alias('count')).sort('user_id')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction_df = grouped_transaction.toPandas()
consumer_transaction = pd.merge(consumer, grouped_transaction_df)
consumer_transaction

22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/05 12:01:01 WARN RowBasedKeyValueBatch: Calling spill() on

,name,address,state,postcode,gender,consumer_id,user_id,avg,sum,count
0,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1195503,1,155.628359,55092.439121,354
1,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,2,149.628090,48629.129236,325
2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,3,171.877445,58610.208770,341
3,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,154128,4,145.245910,50690.822423,349
4,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,712975,5,171.292928,64234.848046,375
...,...,...,...,...,...,...,...,...,...,...
24076,Maria Lopez,6036 Rosario Mount Apt. 283,QLD,4474,Female,831103,24077,139.858120,48950.342123,350
24077,Mark Fischer,31055 Scott Brooks Apt. 953,WA,6566,Male,487249,24078,154.171001,52418.140507,340
24078,Tony Schmidt,670 Burke Turnpike Apt. 079,VIC,3029,Male,256441,24079,188.457799,59929.580041,318
24079,Amy Russo,4525 Frazier Meadows,NSW,2809,Female,940951,24080,165.986721,54775.617949,330
